In [2]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from bitarray import bitarray

importing ngsxfem-2.1.2504


In [3]:
def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [43]:
# physical parameters for linear elastic
mu  = 10
lam = 10

# parameters of DG method
order_u = 1
beta_u = 500 * order_u * order_u

# parameter of ghost penalty
gamma_u = 20

In [44]:
# manufactured solution
u_x = sin(pi*x) * sin(pi*y) 
u_y = cos(pi*x) * sin(pi*y) 

exact_u = CF((u_x, u_y))

# strain tensor
epsilon_xx = u_x.Diff(x)
epsilon_yy = u_y.Diff(y) 
epsilon_xy = 0.5*(u_x.Diff(y) +  u_y.Diff(x))


# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx 
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy 
sigma_xy = 2*mu*epsilon_xy



# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

fe = CF((f_x, f_y))

uD = exact_u

In [45]:
# Construct background mesh
# Geometry and Mesh
h0 = 1/8
quad_mesh = False
square = SplineGeometry()
square.AddRectangle((-1, -1), (1, 1), bc=1)
ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# Draw(mesh)

In [46]:
levelset = sqrt(x**2 + y**2) - 1/2

# Higher order level set approximation
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order_u, threshold=1000,
                                      discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1

# DrawDC(lsetp1,-1,1,mesh)

In [47]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


In [48]:
#Unfitted DG spaces
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for displacement
# U = Restrict(Uhbase,ci.GetElementsOfType(HASNEG))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
u,v = U.TnT()

In [49]:
# Define the jumps and the averages
n = Normalize(grad(lsetp1)) # outer normal vector on the boundary
# n = 1.0 / Norm(grad(lsetp1)) * grad(lsetp1)
ne = specialcf.normal(mesh.dim) # normal vectors on faces
h = specialcf.mesh_size  

strain_u = Sym(Grad(u))
strain_v = Sym(Grad(v))
mean_stress_u = 0.5*(Stress(Sym(Grad(u)))+Stress(Sym(Grad(u.Other()))))*ne
mean_stress_v = 0.5*(Stress(Sym(Grad(v)))+Stress(Sym(Grad(v.Other()))))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [50]:
jump_du = Grad(u)*ne - Grad(u.Other())*ne
jump_dv = Grad(v)*ne - Grad(v.Other())*ne

In [66]:
print(ne.shape)

(2,)


In [65]:
# print(help(ne))

In [51]:
# integration domains:
domega = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
dk = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_facets,
          deformation=deformation)
ds = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation)
# dw = dCut(lsetp1, NEG, skeleton=True, definedonelements=ba_facets,
#           deformation=deformation)
dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)

In [52]:
Ah = BilinearForm(U)
# Ae
Ah += 2*mu*InnerProduct(strain_u,strain_v)*domega + lam*div(u)*div(v)*domega \
        - (InnerProduct(mean_stress_u,jump_v) + InnerProduct(mean_stress_v,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
        - (InnerProduct(Stress(Sym(Grad(u)))*n,v) + InnerProduct(Stress(Sym(Grad(v)))*n,u) - beta_u/h*InnerProduct(u,v))*ds
# order=1 i_s 
# Ah += gamma_u * h * ((Grad(u) - Grad(u.Other()))*ne) * ((Grad(v) - Grad(v.Other()))*ne) * dw
# Ah += gamma_u * h * InnerProduct(Grad(u) - Grad(u.Other()),Grad(v) - Grad(v.Other())) * dw
Ah += gamma_u * h * jump_du * jump_dv * dw

Ah.Assemble()

# r.h.s
lh = LinearForm(U) 
lh += fe*v*domega - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*ds + beta_u/h*uD*v*ds
lh.Assemble()

gfu = GridFunction(U)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

In [53]:
error_u = sqrt(Integrate((gfu - exact_u)**2 * domega, mesh))
print(error_u)

0.00931360025815503


In [54]:
# DrawDC(lsetp1, gfu, (0,0), mesh, "uh", deformation=deformation)
# DrawDC(lsetp1, exact_u, (0,0), mesh, "uh", deformation=deformation)

In [55]:
mask = IfPos(lsetp1,0,1)

In [56]:
# Draw(mask*gfu, mesh)
# Draw(exact_u,mesh,deformation=deformation)
# Draw(mask*exact_u, mesh,deformation=deformation)
Draw(mask*(gfu-exact_u),mesh,deformation=deformation)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()
# positive_values = []
# positive_indices = []
# for v,ind in enumerate(kappaminus_values):
#     positive_values.append(v)
#     positive_indices.append(ind)
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"The smallest cut ratio is: {min_value_pythonic:.2e}")
else:
    print("There are no cut elements.")

In [ ]:
# positive_indices = np.array([v for v in range(len(kappaminus_values)) if kappaminus_values[v] > 0])
positive_indices = np.where(kappaminus_values > 0)[0]
# print(len(positive_values))
sorted_value_indices_in_positive_array = np.argsort(positive_values)
Ke = 10
smallest_K_indices_in_positive_array = sorted_value_indices_in_positive_array[:Ke]
smallest_K_original_indices = positive_indices[smallest_K_indices_in_positive_array]
min_K_values = kappaminus_values[smallest_K_original_indices]
print(f"最小的 {Ke} 个切割单元的比率: {min_K_values}")
print(f"它们在原始数组中的索引: {smallest_K_original_indices}")
print("-" * 30)

# visualize
smallest_K = BitArray(mesh.ne)
smallest_K.Clear()
for i in smallest_K_original_indices:
    smallest_K.Set(i)
Draw(BitArrayCF(smallest_K),mesh)

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# 计算最大奇异值
rows,cols,vals = Ah.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
condition_number = np.linalg.cond(A.todense())
# _, s_max,_ = svds(A, k=1, which='LM')   # largest magnitude

# # # 计算最小奇异值
# _, s_min,_ = svds(A, k=1, which='SM')   # smallest magnitude

# condition_number = s_max[0] / s_min[0]
print(f"{condition_number:.2e}")
